In [1]:
# the code works in Python 3.6.
!pip install simpletransformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/rh/rh-python36/root/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
import sklearn
import pandas as pd 
import io
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs

import warnings
warnings.filterwarnings('ignore')

# input/output data directory
data_dir = f'./data_sample'
out_dir = f'./output'

# the file which stores the predictions 
pred_out_file = f'{out_dir}/preds.csv'

# The positive label for binary classification
pos_label = 1

In [9]:
def load_data(dat_file, is_train):
    global pos_label
    if is_train:
        df= pd.read_csv(dat_file, usecols=['text', 'label']).sample(frac=1).reset_index(drop=True)
    else:
        df= pd.read_csv(dat_file, usecols=['text', 'label'])
    df = df.dropna()
    print(df.head())
    df['label'] = (df['label'] == pos_label).astype(int)
    print(df.head())
    print('size:', len(df))
    return df

In [11]:
df_train = load_data(f'{data_dir}/sample.csv', is_train=True)
df_val = load_data(f'{data_dir}/sample.csv', is_train=False)
df_test = load_data(f'{data_dir}/sample.csv', is_train=False)

                     text  label
0      This is an example      1
1  This is not an example      0
                     text  label
0      This is an example      1
1  This is not an example      0
size: 2
                     text  label
0      This is an example      1
1  This is not an example      0
                     text  label
0      This is an example      1
1  This is not an example      0
size: 2
                     text  label
0      This is an example      1
1  This is not an example      0
                     text  label
0      This is an example      1
1  This is not an example      0
size: 2


In [12]:
# set the model hyperparameters
model_args = ClassificationArgs()

# model arch
model_args.sliding_window = False
model_args.max_seq_length = 512
model_args.train_batch_size = 16
model_args.num_train_epochs = 10
model_args.learning_rate = 2e-05

# input data process
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True

# save checkpoints
model_args.save_model_every_epoch = True
model_args.save_steps = -1
model_args.best_model_dir = f'{out_dir}/best_model'

# early stopping config
#model_args.use_early_stopping = True
#model_args.early_stopping_consider_epochs = True

In [13]:
model = ClassificationModel("roberta", "roberta-base", num_labels=2, use_cuda=True, args=model_args) 

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [14]:
model.train_model(df_train, output_dir=out_dir)

(10, 0.7042608141899109)

In [17]:
# test the model
test_text=list(df_test['text'])
Y_test=df_test['label']
predictions, raw_outputs = model.predict(test_text)

report = classification_report(Y_test, predictions)
f = f1_score(Y_test, predictions)
p = precision_score(Y_test, predictions)
r = recall_score(Y_test, predictions)
print(report)


              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

